In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import sys
import os

In [ ]:
# Definición de rutas para importar y exportar datos repocitorio local

# Definición de rutas para importar y exportar datos repocitorio local // "EN ESTE ESPACIO DEBE IR LA RUTA DONDE DESEA GUARDAR SU PROYECTO."
raiz = r'RUTA DE SU PROYECTO\\Proyecto_ETL_Fuga_de_Cerebros' #<==================DEBES COLOCAR TU RUTA AQUI Pero siempre respetar el \\Proyecto_ETL_Fuga_de_Cerebros'

paths = {
     'path_import': f"{raiz}\\data\\import",
     'path_export': f"{raiz}\\data\\export",

}

print('Paths configurados correctamente')

Paths configurados correctamente


In [8]:
# Guardar en csv

# === CONFIGURACIÓN DE RUTAS y llamado de data ===

#Configuración de rutas de archivos
recopilacion_promedios = os.path.join(paths['path_export'], 'Scraping_salarios_colombia.xlsx')
agrup_promedios = os.path.join(paths['path_export'], 'Agrup_datos_salariales.xlsx')
# Nombre del archivo de entrada
archivo_salida1 = recopilacion_promedios
archivo_salida2 = agrup_promedios


In [ ]:

#Implementación de Categorias para el filtro // EJECUCIÓN DE SCRAPING.

categoria =[
    "administracion-y-oficina",
    "almacen-logistica",
    "atencion-a-clientes",
    "callcenter-telemercadeo",
    "compras-comercio-exterior",
    "construccion-y-obra",
    "economia-y-contabilidad",
    "direccion-y-gerencia",
    "arte-y-diseno-y-medios",
    "educacion-y-universidad",
    "hosteleria-y-turismo",
    "informatica-y-telecom",
    "ingenieria-y-tecnico",
    "cientifico-y-investigacion",
    "legal-y-asesoria",
    "mantenimiento-y-reparaciones-tecnicas",
    "medicina-y-salud",
    "mercadeo-publicidad-comunicacion",
    "produccion-operaciones",
    "recursos-humanos",
    "servicios-generales-aseo-y-seguridad",
    "marketing-y-ventas"
]


# Función para scrapear una categoría completa
def scrap_categoria(categoria, max_pages=50):

  url =f'https://co.computrabajo.com/salarios-de-{categoria}?by=averageDesc&p={{}}'
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
            }
  profesiones = []

  # Recorremos páginas (ajusta el rango si quieres más)
  for page in range(1, max_pages+50):
      res = requests.get(url.format(page), headers=headers)
      soup = BeautifulSoup(res.text, "html.parser")

      #Revisión Fila x Fila teniendo en cuenta la clave de estructura de "div.dTable.w_100.pb40.row"
      filas = soup.select("div.dTable.w_100.pb40.row")

      for fila in filas:
          # Buscamos todos los bloques de filas
          puestos = fila.select_one("div.tablec.w_40.vmid.fw_b.fs16 a") or fila.select_one("div.tablec.w_40.vmid.fw_b.fs16 span")
          salarios = fila.select_one("div.tablec.w_30.vmid p.fw_b.fs18")
          n_salarios = fila.select_one("div.tablec.w_30.vmid span.fc80")
          rangos_salariales = fila.select_one("div.dFlex.tj_fx.fs12.fc_aux.mt5")

          #Condicional para verificación o salteo de dato en caso de no tener una estructura adecuada en base a la variable "Puestos"
          if not puestos or not salarios:
            continue # Las variables de puestos y salarios se encuentran vacios

          salario_min = None
          salario_max = None

## Ejempo de (get_text)
#from bs4 import BeautifulSoup
#html = "<p>Hola <b>Mundo</b></p>"
#soup = BeautifulSoup(html, "html.parser")
#parrafo = soup.find("p")
#print(parrafo.get_text())
# SALIDA DEL PRINT===================================== "Hola Mundo"


          if rangos_salariales:
            valores = [x.get_text(strip=True)for x in rangos_salariales.select("p")]
            if len(valores)>0:
              salario_min = valores[0]
            if len(valores)>1:
              salario_max = valores [1]


          profesiones.append({
              "CATEGORIA": categoria,
              "puesto": puestos.get_text(strip=True),
              "salario": salarios.get_text(strip=True),
              "n_salarios": n_salarios.get_text(strip=True),
              "salario_min": salario_min,
              "salario_max": salario_max
            })

      time.sleep(1) #Delay de 1 segúndo por la carga de cada pagina.

  return profesiones #devolucion de la lista por categoria

#Almacenamiento de todos los datos
# ---- Scrapeamos TODAS las categorías ----
todos_datos=[]
for cat in categoria:
  print(f'Scrapeando categoria: {cat}')
  datos_cat= scrap_categoria(cat, max_pages=2)
  todos_datos.extend(datos_cat)

df = pd.DataFrame(todos_datos)
print("Total profesiones encontradas:", len(df))
print(df.head(162))

# Exportar DataFrame a archivo CSV
df.to_csv(archivo_salida1, index=False)
print("Archivo generado y guardado.")


Scrapeando categoria: administracion-y-oficina
Scrapeando categoria: almacen-logistica
Scrapeando categoria: atencion-a-clientes
Scrapeando categoria: callcenter-telemercadeo
Scrapeando categoria: compras-comercio-exterior
Scrapeando categoria: construccion-y-obra
Scrapeando categoria: economia-y-contabilidad
Scrapeando categoria: direccion-y-gerencia
Scrapeando categoria: arte-y-diseno-y-medios
Scrapeando categoria: educacion-y-universidad
Scrapeando categoria: hosteleria-y-turismo
Scrapeando categoria: informatica-y-telecom
Scrapeando categoria: ingenieria-y-tecnico


KeyboardInterrupt: 

In [ ]:
# Proceso de agrupación y cálculo de promedios
# Función para limpiar y convertir el salario a un número entero

def limpiar_salario(valor):
    if pd.isna(valor):
        return None

    # Quitamos el símbolo $ y el texto '/mes'
    valor = valor.replace("$", "").replace("/mes", "").strip()

    # Quitamos los puntos separadores de miles
    valor = valor.replace(".", "")

    # Si queda vacío, devolvemos None
    if valor == "":
        return None

    return int(valor)

# Aplicamos la limpieza
df["salario_limpio"] = df["salario"].apply(limpiar_salario)

# Agrupamos y calculamos el promedio por categoría
df_promedio = (
    df.groupby("CATEGORIA", as_index=False)["salario_limpio"]
    .mean()
    .rename(columns={"salario_limpio": "PROMEDIO_SALARIAL"})
)
df_promedio["CATEGORIA"] = df_promedio["CATEGORIA"].str.replace("-", " ")


print(df_promedio)

#Exportar DataFrame a archivo CSV

df_promedio.to_csv(archivo_salida2, index=False)
print("Archivo generado y guardado.")

In [ ]:

#___________________________Carga de datos a BASE DE MYSQL_____________________________________
# import mysql.connector
# from mysql.connector import Error

# #data = [tuple(x) for x in df.values.tolist()]
# data = [tuple(None if pd.isna(xi) else xi for xi in x) for x in df_promedio.values.tolist()]


# try:
#     connection = mysql.connector.connect(
#         host='localhost',
#         port=3306,
#         database='promedio_salarial',
#         user='root',
#         password='',
#         auth_plugin='mysql_native_password'
#     )

#     if connection.is_connected():
#         cursor = connection.cursor()

#         batch_size = 1000
#         for i in range(0, len(data), batch_size):
#             cursor.executemany(
#                 """INSERT INTO categ_promedi_salarial 
#                 (categoria,promedio_salarial)
#                 VALUES (%s,%s)""", 
#                 data[i:i+batch_size]
#             )
#             connection.commit()

#         print(f"{cursor.rowcount} registros insertados")

# except Error as ex:
#     print("Error while connecting to MySQL", ex)

# finally:
#     if connection.is_connected():
#         connection.close()
#         print("Connection Closed")
#______________________________________________________________________________________________